<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Data_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
result = joblib.load(f"/content/4D-Humans/outputs/results/demo_{video_name}.pkl")
with open("/content/result_output.txt", "w") as file:
    file.write(str(result))

In [ ]:
import numpy as np
import joblib
import plotly.graph_objects as go

# Lade die Daten
result = joblib.load(f"/content/4D-Humans/outputs/results/demo_{video_name}.pkl")
frame_key = list(result.keys())[0]  # Nimm den ersten Frame
joints_3d = np.array(result[frame_key]['3d_joints'][0])  # 3D-Joints der ersten Person

# x, y und z
x = joints_3d[:, 0]
y = joints_3d[:, 1]
z = joints_3d[:, 2]

# Rotation durchführen, damit das Modell richtig ausgerichtet wird
# Dies ist eine 180-Grad-Drehung um die X-Achse
rotation_matrix = np.array([[1, 0, 0],
                            [0, -1, 0],  # Spiegeln der Y-Achse
                            [0, 0, -1]]) # Spiegeln der Z-Achse, damit die Person richtig steht

# Anwenden der Rotation auf die Gelenk-Punkte
joints_rotated = np.dot(np.column_stack([x, y, z]), rotation_matrix.T)

# Die neuen x, y, z Werte nach der Rotation
x_rot = joints_rotated[:, 0]
y_rot = joints_rotated[:, 1]
z_rot = joints_rotated[:, 2]

# Plotly 3D Scatter Plot erstellen
fig = go.Figure()

# Punkte plotten
fig.add_trace(go.Scatter3d(
    x=x_rot,
    y=y_rot,
    z=z_rot,
    mode='markers+text',  # Punkte und Text anzeigen
    marker=dict(size=5, color='red'),  # Marker-Einstellungen: rote Farbe und Größe 5
    text=[str(i) for i in range(len(x_rot))],  # Index als Text neben jedem Punkt
    textposition="top center",  # Text oberhalb der Punkte
    name="Joints"  # Name für die Legende
))

# Finde den maximalen Wert für x, y und z, um die gleiche Skala zu setzen
max_range = np.max([np.abs(x_rot).max(), np.abs(y_rot).max(), np.abs(z_rot).max()])

# Achsenbeschriftungen und Titel setzen
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(range=[-max_range, max_range]),
        yaxis=dict(range=[-max_range, max_range]),
        zaxis=dict(range=[-max_range, max_range]),
        camera=dict(
            eye=dict(x=0, y=0, z=2),  # Perspektive so anpassen, dass der Mensch aufrecht ist
            up=dict(x=0, y=1, z=0),   # Vertikale Ausrichtung auf Y
            center=dict(x=0, y=0, z=0)  # Mittelpunkt des Koordinatensystems
        ),
    ),
    title="3D Joint Visualization"
)

# Plot anzeigen
fig.show()